# FIT5202 Data processing for Big data

##  Activity: Assignment 2 Part B

### Task 1

##### Student ID: `31265154`
##### Student Name: `Vivekkumar Chaudhari`

<h3>About Producer</h3><p>It creates the Kafka topic, and send pedestrian CSV data for December 2020.&nbsp;</p>

In [1]:
#Importing Requires libraries
from time import sleep
from kafka import KafkaProducer
from datetime import datetime as dt
import pandas as pd

# To publish message(data) on kafka topic with given Kafka Producer.
def publish_message(producer_instance, topic_name, data):
    try:
        data_bytes = bytes(str(data), encoding='utf-8')
        producer_instance.send(topic_name, data_bytes)
        producer_instance.flush()
        print(f'Successfully sent data: {data}')
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

# It will try to establish connection with given location and port to get Kafka Producer.
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

# Main execution flow
if __name__ == '__main__':
    topic = 'Pedestrian_data'
    print('Publishing records..')
    producer = connect_kafka_producer()
    # Read csv data to panda dataframe
    df = pd.read_csv(
        filepath_or_buffer='Streaming_Pedestrian_December_counts_per_hour.csv', 
        sep=',',
        usecols=[
            'Mdate', 'ID', 'Date_Time', 'Year', 
            'Month', 'Day', 
            'Time', 'Sensor_ID', 'Sensor_Name', 'Hourly_Counts'],
        dtype={
            'ID': int, 'Date_Time': str, 'Year': int, 
            'Month': str, 'Mdate': int, 'Day': str, 
            'Time': int, 'Sensor_ID': int, 'Sensor_Name': str, 'Hourly_Counts': int}
    )
    # Group data over daywise
    df_daywise = df.groupby('Mdate')
    day_counter = 1
    count = 0
    # send data daywise in batch
    while day_counter <= len(df_daywise):
        value = df_daywise.get_group(day_counter).to_dict(orient='records')
        publish_message(producer, topic, value)
        day_counter += 1
        sleep(5)

Publishing records..
Successfully sent data: [{'ID': 3435630, 'Date_Time': '12/01/2020 08:00:00 AM', 'Year': 2020, 'Month': 'December', 'Mdate': 1, 'Day': 'Tuesday', 'Time': 8, 'Sensor_ID': 39, 'Sensor_Name': 'Alfred Place', 'Hourly_Counts': 83}, {'ID': 3435798, 'Date_Time': '12/01/2020 11:00:00 AM', 'Year': 2020, 'Month': 'December', 'Mdate': 1, 'Day': 'Tuesday', 'Time': 11, 'Sensor_ID': 12, 'Sensor_Name': 'New Quay', 'Hourly_Counts': 86}, {'ID': 3435107, 'Date_Time': '12/01/2020 12:00:00 AM', 'Year': 2020, 'Month': 'December', 'Mdate': 1, 'Day': 'Tuesday', 'Time': 0, 'Sensor_ID': 4, 'Sensor_Name': 'Town Hall (West)', 'Hourly_Counts': 78}, {'ID': 3435108, 'Date_Time': '12/01/2020 12:00:00 AM', 'Year': 2020, 'Month': 'December', 'Mdate': 1, 'Day': 'Tuesday', 'Time': 0, 'Sensor_ID': 17, 'Sensor_Name': 'Collins Place (South)', 'Hourly_Counts': 10}, {'ID': 3435109, 'Date_Time': '12/01/2020 12:00:00 AM', 'Year': 2020, 'Month': 'December', 'Mdate': 1, 'Day': 'Tuesday', 'Time': 0, 'Sensor_ID

KeyboardInterrupt: 